In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(),os.pardir))# TODO: change when change to python file
import pandas as pd
import torch
import recipe_1m_analysis.utils as utils
import numpy as np
import math
import matplotlib.pyplot as plt
from datetime import datetime
import csv

from sklearn import metrics
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader, Sampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
import pickle

## Data preprocessing

In [32]:
file = os.path.join(os.pardir,"recipe_1m_analysis","data","recipe1m_train_cuisine.pkl")
with open(file,"rb") as f:
    data = pickle.load(f)

In [37]:
for recipe in data.values():
    if "cuisine" in recipe:
        print(recipe)

In [33]:
df = pd.DataFrame.from_dict(data, orient='index')
df

,tokenized,instructions,ingredients,title
feb4fe6dbb,"[[preheat, oven, to, 400, ;, spray, a, 13, x, ...",[preheat oven to 400; spray a 13 x 9 inch oven...,"[(butter, (None, None, None, None), False), (o...","[zucchini, pie, with, parmesan, rice, crust]"
d2c3d153b3,"[[in, a, large, frying, pan, ,, cook, bacon, s...","[in a large frying pan, cook bacon slices unti...","[(bacon, (None, None, None, None), False), (on...","[hot, and, spicy, black-eyed, peas]"
b725b5fc36,"[[in, small, bowl, ,, combine, the, flour, ,, ...","[in small bowl, combine the flour, ketchup, wo...","[(flour, (None, None, None, None), False), (ke...","[grilled, sirloin, roast]"
f60379e25d,"[[measure, ingredients, into, baking, pan, in,...",[measure ingredients into baking pan in the or...,"[(water, (None, None, None, None), False), (no...","[onion, rye, loaf, (, bread, machine, )]"
945be4ed6b,"[[preheat, oven, at, 350f, (, 180c, ), .], [me...","[preheat oven at 350f (180c)., melt the butter...","[(puff_pastry, (None, None, None, None), False...","[ground, sirloin, wellington]"
...,...,...,...,...
6d5d5fc27e,"[[in, a, blender, ,, combine, peanut, butter, ...","[in a blender, combine peanut butter, chocolat...","[(creamy_peanut_butter, (None, None, None, Non...","[chocolate, peanut, butter, milkshake]"
76a2f7b132,"[[combine, tahini, and, lemon, juice, and, ble...",[combine tahini and lemon juice and blend unti...,"[(chickpea, (None, None, None, None), True), (...","[rockin, ', hummus]"
8deff436e3,"[[line, a, bamboo, steamer, with, banana, leav...","[line a bamboo steamer with banana leaves, or ...","[(halibut_fillet, (None, None, None, None), Tr...","[chile-ginger, halibut]"
9a34bd4a6c,"[[season, pork, chops, with, cajun, seasoning,...","[season pork chops with cajun seasoning., add ...","[(boneles_pork_chop, (None, None, None, None),...","[crock, pot, pork, stroganoff]"


In [31]:
df = df.reset_index()
df["index"]

0         feb4fe6dbb
1         d2c3d153b3
2         b725b5fc36
3         f60379e25d
4         945be4ed6b
             ...    
113350    6d5d5fc27e
113351    76a2f7b132
113352    8deff436e3
113353    9a34bd4a6c
113354    f44a8cfc26
Name: index, Length: 113355, dtype: object

In [22]:
df.index.values.tolist()

113355

In [23]:
for ingrs,label in zip(df["ingredients"].items(),df.index):
    #self.data[idx]=(torch.LongTensor(self.ingr2idx(ingrs)),label[1])
    print(ingrs,label)
    if idx==10:#00: #TODO: to remove when running smoothly
        break

('feb4fe6dbb', [Ingredient(name='butter', amount=(None, None, None, None), plural=False), Ingredient(name='onion', amount=(None, None, None, None), plural=True), Ingredient(name='garlic_clove', amount=(None, None, None, None), plural=True), Ingredient(name='zucchini', amount=(None, None, None, None), plural=False), Ingredient(name='salt', amount=(None, None, None, None), plural=False), Ingredient(name='black_pepper', amount=(None, None, None, None), plural=False), Ingredient(name='tabasco_sauce', amount=(None, None, None, None), plural=False), Ingredient(name='-_flour', amount=(None, None, None, None), plural=False), Ingredient(name='vegetable_broth', amount=(None, None, None, None), plural=False), Ingredient(name='heavy_cream', amount=(None, None, None, None), plural=False), Ingredient(name='long_-_grain_white_rice', amount=(None, None, None, None), plural=False), Ingredient(name='parmesan_cheese', amount=(None, None, None, None), plural=False)]) feb4fe6dbb


NameError: name 'idx' is not defined

In [16]:
for i,row in enumerate(df.iterrows()):
    print(i,row[0],[ingr.name for ingr in row[1]["ingredients"]])
    if i==10:
        break

0 feb4fe6dbb ['butter', 'onion', 'garlic_clove', 'zucchini', 'salt', 'black_pepper', 'tabasco_sauce', '-_flour', 'vegetable_broth', 'heavy_cream', 'long_-_grain_white_rice', 'parmesan_cheese']
1 d2c3d153b3 ['bacon', 'onion', 'green_bell_pepper', 'garlic_clove', 'black_-_eyed_pea', 'diced_tomato', 'ground_cumin', 'salt', 'curry_powder', 'cayenne_pepper', 'chili_powder']
2 b725b5fc36 ['flour', 'ketchup', 'worcestershire_sauce', 'brown_sugar', 'cider_vinegar', 'salt', 'prepared_mustard', 'pepper', 'boneles_sirloin_tip_roast', 'baby_carrot', 'tomato', 'onion']
3 f60379e25d ['water', 'nonfat_dry_milk_powder', 'salt', 'brown_sugar', 'vegetable_oil', 'bread_flour', 'rye_flour', 'dried_onion_flake', 'bread_machine_yeast']
4 945be4ed6b ['puff_pastry', 'butter', 'mushroom', 'onion', 'sirloin', 'parsley', 'basil', 'salt', 'pepper', 'egg', 'liver_pate']
5 9f0a2d381a ['lemongras', 'salt', 'red_pepper', 'turmeric']
6 0a3406ab20 ['boneles_skinles_chicken_breast', 'onion', 'celery', 'green_bell_pepper

In [2]:
FOLDER_PATH = "D:\\Google Drive\\Catherning Folder\\THU\\Thesis\\Recipe datasets\\"
DATASET = ["scirep-cuisines-detail","Yummly28"]
FILES = ["random","cluster_centroid","full"]

file = FILES[2]
balanced = False
load = False

In [4]:
dataset = DATASET[1]

df=pd.read_pickle(os.path.join(FOLDER_PATH,dataset,file+"_data.pkl"))
df=df.set_index("id")

In [5]:
dataset = DATASET[0]

df2=pd.read_pickle(os.path.join(FOLDER_PATH,dataset,file+"_data.pkl"))
df2["id"]=[len(df)+i for i in range(len(df2))]
df2=df2.set_index("id")
df = pd.concat([df, df2])
df

# TODO: Try by only selecting recipes where ingredients are in vocab_ingr from recipe1M ?

,cuisine,ingredients,all_ingredients
id,,,
0,Italian,"[baby bella mushroom sliced, arborio rice, oli...",baby bella mushroom sliced;arborio rice;olive ...
2,Italian,"[garlic, extra-virgin olive oil divided, madei...",garlic;extra-virgin olive oil divided;madeira ...
3,French,"[garlic halved, cooking spray, peeled red pota...",garlic halved;cooking spray;peeled red potato;...
5,American,"[turkey thawed, maple syrup, bourbon, pickling...",turkey thawed;maple syrup;bourbon;pickling spi...
6,American,"[beef tenderloin silver skin trimmed, worceste...",beef tenderloin silver skin trimmed;worcesters...
...,...,...,...
84008,Italian,"[kiwi, olive_oil, clam, white_wine, orange, sa...",kiwi;olive_oil;clam;white_wine;orange;salmon;f...
84009,Italian,"[tomato, butter, beef, onion, red_wine, black_...",tomato;butter;beef;onion;red_wine;black_pepper...
84010,Italian,"[vegetable, wheat, egg, cheese, olive_oil]",vegetable;wheat;egg;cheese;olive_oil


In [6]:
vocab_ingrs = utils.Vocabulary()
for ingredients in df.loc[:,"ingredients"]:
    for ingr in ingredients:
        vocab_ingrs.add_word(ingr)
vocab_ingrs.add_word("<unk>")
vocab_ingrs.word2idx

{'baby bella mushroom sliced': 0,
 'arborio rice': 1,
 'olive oil': 2,
 'butter': 3,
 'shallot minced': 4,
 'white wine': 5,
 'fat free chicken stock': 6,
 'salt pepper': 7,
 'grated parmesan cheese': 8,
 'chopped parsley': 9,
 'garlic': 10,
 'extra-virgin olive oil divided': 11,
 'madeira divided': 12,
 'unsalted chicken stock divided': 13,
 'dried porcini mushroom': 14,
 'chopped onion': 15,
 'thinly sliced cremini mushroom': 16,
 'thinly sliced shiitake mushroom cap': 17,
 'uncooked arborio rice': 18,
 'parmesan cheese grated': 19,
 'kosher salt': 20,
 'freshly ground black pepper': 21,
 'chopped fresh sage': 22,
 'sage': 23,
 'garlic halved': 24,
 'cooking spray': 25,
 'peeled red potato': 26,
 'butter melted': 27,
 'salt': 28,
 'black pepper': 29,
 'shredded gruyere cheese': 30,
 'fat-free milk': 31,
 'turkey thawed': 32,
 'maple syrup': 33,
 'bourbon': 34,
 'pickling spice': 35,
 'large carrot scraped': 36,
 'celery rib': 37,
 'medium onion peeled halved': 38,
 'lemon': 39,
 'pep

In [7]:
vocab_cuisine = utils.Vocabulary()
for cuisine in df['cuisine'].value_counts().index:
    vocab_cuisine.add_word(cuisine)
vocab_cuisine.word2idx

{'American': 0,
 'Italian': 1,
 'Mexican': 2,
 'Asian': 3,
 'French': 4,
 'Indian': 5,
 'east_asian': 6,
 'Korean': 7,
 'Canada': 8,
 'western': 9,
 'Chinese': 10,
 'Mediterranean': 11,
 'Spanish_Portuguese': 12,
 'Eastern-Europe': 13,
 'Southern_SoulFood': 14,
 'Jewish': 15,
 'Japanese': 16,
 'Germany': 17,
 'Thai': 18,
 'UK-and-Ireland': 19,
 'Scandinavian': 20,
 'MiddleEastern': 21,
 'Central_SouthAmerican': 22,
 'Greek': 23,
 'English_Scottish': 24,
 'Caribbean': 25,
 'Moroccan': 26,
 'Cajun_Creole': 27,
 'African': 28,
 'Southwestern': 29,
 'South-America': 30,
 'Vietnamese': 31,
 'Irish': 32,
 'North-African': 33,
 'Portugal': 34,
 'Philippines': 35,
 'Netherlands': 36,
 'Lebanon': 37,
 'Cuban': 38,
 'Austria': 39,
 'Iran': 40,
 'Switzerland': 41,
 'Pakistan': 42,
 'Malaysia': 43,
 'asian': 44,
 'South-African': 45,
 'Turkey': 46,
 'English_Irish': 47,
 'West-African': 48,
 'Indonesia': 49,
 'Belgium': 50,
 'East-African': 51}

## Data processing

In [8]:
def ingr2idx(ingr_list):
    # If I didn't do the one-hot encoding by myself and used directly an embedding layer in the net, 
    # I would have to pad the input
    input_=[]
    for ingr in ingr_list:
        try:
            input_.append(vocab_ingrs.word2idx[ingr])
        except KeyError:
            input_.append(vocab_ingrs.word2idx["<unk>"])
    input_ = torch.LongTensor(input_)
    onehot_enc = F.one_hot(input_.to(torch.int64), INPUT_SIZE)
    output = torch.sum(onehot_enc,0)
    return output

class IngrDataset(Dataset):
    """Recipes dataset for cuisine classification. Only from ingredients for now"""

    def __init__(self, input_,labels):
        """
        Args:
            file (string): Path to the file
        """
        self.input_ = input_
        self.labels = labels

    def __len__(self):
        return len(self.input_)

    def __getitem__(self, idx):
        # XXX useful ?
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        ingr_one_hot = ingr2idx(self.input_.loc[idx,"ingredients"])
        label = vocab_cuisine.word2idx[self.labels.loc[idx,"cuisine"]]
        return ingr_one_hot, label
        

In [9]:
def make_weights_for_balanced_classes(samples, nclasses): 
    count = [0] * nclasses
    weight_per_class = [0.] * nclasses
    N = len(samples)
    
    for a,el in samples.value_counts().items():
        count[vocab_cuisine.word2idx[a]]=el

    for i in range(nclasses): # XXX to still sample the others, add log ? so goes back to N, not max(count)
        weight_per_class[i] = math.log(N/float(count[i])) # divide by max count[i] ? Or just different scale, order is same
        print(vocab_cuisine.idx2word[i], weight_per_class[i])
    weight = [0] * N
    
    for idx, val in enumerate(samples): 
        weight[idx] = weight_per_class[vocab_cuisine.word2idx[val]] 
        
    
    return torch.Tensor(weight_per_class), torch.DoubleTensor(weight)

# from https://gist.github.com/srikarplus/15d7263ae2c82e82fe194fc94321f34e

In [32]:
INPUT_SIZE = len(vocab_ingrs)
EMBED_DIM1 = 300
EMBED_DIM2 = 64
NUM_CLASSES = len(vocab_cuisine) #51 or 20
BATCH_SIZE = 100
PRINT_FREQ = 20
NB_EPOCHS = 30

In [ ]:
date = datetime.now().strftime("%m-%d-%H-%M")
RESULTS_FOLDER = os.path.join(os.getcwd(),"results",f"nn_{date}{balanced*'_bal'}_{file}")
if not os.path.exists(RESULTS_FOLDER):
    os.makedirs(RESULTS_FOLDER)

Weighted random sampling, with stratified split for the train and test dataset. But loss doesn't decrease (need to see more epochs ?)

In [ ]:
#TODO when switch to python file, can put num_workers & have to put if __name__ == '__main__':
X_train, X_dev, y_train, y_dev = train_test_split(df["ingredients"],df["cuisine"], test_size=0.2, random_state=42,stratify=df["cuisine"])
X_dev, X_test, y_dev, y_test = train_test_split(X_dev,y_dev, test_size=0.5, random_state=42,stratify=y_dev)

X_train = X_train.reset_index()
X_dev = X_dev.reset_index()
X_test = X_test.reset_index()
y_train = y_train.reset_index()
y_dev = y_dev.reset_index()
y_test = y_test.reset_index()

train_dataset = IngrDataset(X_train,y_train)
dev_dataset = IngrDataset(X_dev,y_dev)
test_dataset = IngrDataset(X_test,y_test)

if balanced:
    weights_classes, weights_labels = make_weights_for_balanced_classes(y_train["cuisine"], len(vocab_cuisine.word2idx)) 
    print(len(weights_labels))
    sampler = torch.utils.data.sampler.WeightedRandomSampler(weights_labels, len(weights_labels)) 
    train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE, sampler = sampler)#, pin_memory=True)
else:
    train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE)#, pin_memory=True)    

dev_loader = DataLoader(dev_dataset,batch_size = 1)
test_loader = DataLoader(test_dataset, batch_size=1)#, sampler = sampler)

# Model

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim1, embedding_dim2, num_classes):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(vocab_size, embedding_dim1, bias=True)
        self.layer_2 = nn.Linear(embedding_dim1, embedding_dim1, bias=True)
#         self.layer_3 = nn.Linear(embedding_dim1, embedding_dim2, bias=True)
        self.output_layer = nn.Linear(embedding_dim1, num_classes, bias=True)

    def forward(self, x):
        out = F.relu(self.layer_1(x))
        out = F.relu(self.layer_2(out))
#         out = F.relu(self.layer_3(out))
        out = self.output_layer(out)
        return out

net = Net(INPUT_SIZE, EMBED_DIM1, EMBED_DIM2, NUM_CLASSES)

## Training

In [ ]:
def f2_score(y_true, y_pred, threshold=0.5):
    return fbeta_score(y_true, y_pred, 2, threshold)

def fbeta_score(y_true, y_pred, beta, threshold, eps=1e-9):
    beta2 = beta**2

    y_pred = torch.ge(y_pred.float(), threshold).float()
    y_true = y_true.float()

    true_positive = (y_pred * y_true).sum(dim=1)
    precision = true_positive.div(y_pred.sum(dim=1).add(eps))
    recall = true_positive.div(y_true.sum(dim=1).add(eps))

    return torch.mean(
        (precision*recall).
        div(precision.mul(beta2) + recall + eps).
        mul(1 + beta2))

In [ ]:
def score(network, dataloader,test=False,threshold=None):
    network.eval()
    count_unk=0
    correct = 0
    total = 0
    all_predict = []
    all_labels = []
    with torch.no_grad():
        for data in dataloader:
            inputs = data[0]

            # Removing samples where you don't know more than 2 of the ingr doesn't help the model much
            if inputs[0][vocab_ingrs.word2idx["<unk>"]]>3:
                count_unk+=1
                continue

            labels = data[1]
            outputs = network(inputs.float())
            
            if test:
                # Only taking the prediction when the model thinks it's threshold% probable that the label is x
                # Also not that good, accuracy of 42% instead of 82% on dev set 
                proba = torch.nn.functional.softmax(outputs,dim=1)
                p_max, predicted = torch.max(proba,1)
                if p_max < threshold:
                    continue
            else:
                _, predicted = torch.max(outputs.data, 1)
            
            all_predict.append(predicted)
            all_labels.append(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy= 100 * correct / total
    print(f'Accuracy of the network on the test dataset: {accuracy:.3f}% for {total} samples')

    one_hot_pred = F.one_hot(torch.LongTensor(all_predict).to(torch.int64), NUM_CLASSES)
    one_hot_lab = F.one_hot(torch.LongTensor(all_labels).to(torch.int64), NUM_CLASSES)
    fbeta_pytorch = f2_score(one_hot_pred, one_hot_lab)

    print(f'Score is {100* fbeta_pytorch:.3f}%')
    print(f'Count unknown ingr: {count_unk}')
    
    return accuracy, fbeta_pytorch

In [ ]:
if balanced:
    criterion = nn.CrossEntropyLoss(weights_classes)
else:
    criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01) #change to Adam ?

In [ ]:
if load:
    net.load_state_dict(torch.load(os.path.join(FOLDER_PATH,DATASET[1],"model_logweights")))
else:
    epoch_accuracy = []
    epoch_test_accuracy = []
    best_score = 0

    for epoch in range(NB_EPOCHS):
        net.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data[0]
            labels = data[1]
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # print statistics
            running_loss += loss.item()
            if i % PRINT_FREQ == PRINT_FREQ-1:    # print every 2000 mini-batches
                print(f'[Epoch {epoch + 1}, Iteration {i + 1}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

            accuracy = 100 * correct / total
        epoch_accuracy.append(accuracy)
        
        print(f'Accuracy of the network on epoch {epoch+1}: {accuracy:.3f}')
        
        dev_accuracy, dev_fscore = score(net,dev_loader)
        epoch_test_accuracy.append(dev_fscore)
        if dev_fscore > best_score:
            best_score = dev_fscore
            torch.save(net.state_dict(), os.path.join(RESULTS_FOLDER,"best_model"))

    print('Finished Training')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x_points = [i for i in range(0,NB_EPOCHS)]

p = ax.plot(x_points, epoch_accuracy, 'b')
p2 = ax.plot(x_points, epoch_test_accuracy, 'b')
ax.set_xlabel('epoch')
ax.set_ylabel('accuracy')
ax.set_title('Accuracy during training')
fig.show()

fig.savefig(os.path.join(RESULTS_FOLDER,'accuracy_training.png'), dpi=fig.dpi)

## Testing on dev set

In [ ]:
dev_accuracy, dev_fscore = score(net,dev_loader)

## On test set

In [ ]:
threshold = 0.90
test_accuracy, test_fscore = score(net,test_loader,test=True,threshold=threshold)

In [ ]:
torch.save(net.state_dict(), os.path.join(RESULTS_FOLDER,"model_logweights"))

torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, os.path.join(RESULTS_FOLDER,"training_state_logweights"))

In [ ]:
results_file = os.path.join(RESULTS_FOLDER,"results.csv")
if os.path.isfile(results_file):
    with open(results_file,"w", newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(["date","file","balanced","epoch","train_accuracy","dev_accuracy","dev_fscore","test_accuracy","test_fscore","threshold"])
        writer.writerow([date,file,balanced,NB_EPOCHS,epoch_accuracy[-1],dev_accuracy,dev_fscore,test_accuracy,test_fscore,threshold])
else:
    with open(results_file,"a", newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow([date,file,balanced,NB_EPOCHS,epoch_accuracy[-1],dev_accuracy,dev_fscore,test_accuracy,test_fscore,threshold])